# Reading dataset from Google Drive

In [103]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file  
downloaded = drive.CreateFile({'id':'1oAnEjnriYYAJ2kLcsmhVndoNYI6SIcMg'})
downloaded.GetContentFile('train.csv')  

#3. Read file as panda dataframe4/AADZv6Zwa5VmEzlzfS0VGMvvfZqDUF3IuOdlWh26SVI5qPjvYyD8YV0
import pandas as pd
df = pd.read_csv('train.csv')
y_tr = df.pop('label')
print(df.columns)

#downloaded = drive.CreateFile({'id':'1ynUlycQI0gp7yqKzMHdeCX6x-npVDvQZ'})
#downloaded.GetContentFile('GoogleNews-vectors-negative300.bin')  

Index(['id', 'title', 'author', 'text'], dtype='object')


In [0]:
# print (df)
#print(df.shape)
#print(df.columns)
#print(df.head())

# Instaling Gensim and importing libraries

In [0]:
!pip install -q gensim

In [0]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.corpora import Dictionary
from gensim.models import ldamodel
from gensim import corpora
%matplotlib inline

In [33]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> tokenizers/punkt/PY3/english.pickle
    Error loading tokenizers/punkt/PY3/english.pickle: Package
        'tokenizers/punkt/PY3/english.pickle' not found in index

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> english.pickle
Command 'english.pickle' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> 

True

# Data Cleaning (Pre-processing)

In [0]:
n_txt=df['text']
n_txt_lis=n_txt.tolist()
n_txt_tkn=[]
for lis in n_txt_lis:
  lis=str(lis)
  n_txt_tkn.append(lis)
documents = [re.sub("[^a-zA-Z]+", " ", text) for text in n_txt_tkn]
data = [[word for word in text.lower().split() ] for text in documents]
lmtzr = WordNetLemmatizer()
data = [[lmtzr.lemmatize(word) for word in text ] for text in data]

In [0]:
comm_words = stopwords.words('english')
data = [[word for word in text if word not in comm_words] for text in data]
data = [[ word for word in tokens if len(word) >= 3 ] for tokens in data]

# Word2Vec for converting words to vector representation

In [0]:
from nltk.corpus import brown

import gensim.models.word2vec as w2v
model = w2v.Word2Vec(brown.sents(),size=50, window=8)

In [0]:
v = model.wv.vocab

In [110]:
import numpy as np
x_tr_ma=[]
def build_avg_vec(news):
    
    feature_ma=[]
    featureVec = np.zeros((50), dtype="float32")
    nwords=0
    for word in news :
        if(nwords==100):
          break
        nwords += 1
        if word in v:
            featureVec = np.add(featureVec, model[word])
            vcc=model[word]
            vcc.reshape(-1,1)
            feature_ma.append(vcc)
        else:
            vcc=np.zeros(50)
            vcc.reshape(-1,1,1)
            feature_ma.append(vcc)
    
    if(nwords<100):
      while nwords<100:
        nwords+=1
        vcc=np.zeros(50)
        vcc.reshape(-1,1,1)
        feature_ma.append(vcc)
      
    feature_ma=np.array(feature_ma)
    
    if(nwords>0):
        featureVec = np.divide(featureVec, nwords)
    x_tr_ma.append(feature_ma)
    
    return featureVec

x_tr=[]

for i in range (len(data)):
  vec=build_avg_vec(data[i])
  x_tr.append(vec)
   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
#@title Default title text
x_tr_ma=np.array(x_tr_ma)
x_tr_ma = x_tr_ma.reshape(-1,100,50,1)
#print(x_tr_ma.shape)

In [0]:
x_tr_ma=np.array(x_tr_ma)
y_tr=np.array(y_tr)

from sklearn.cross_validation import train_test_split
x_trr, x_te, y_trr, y_te = train_test_split(x_tr_ma, y_tr, test_size=0.20, random_state=0)
x_tr_s,x_te_s,y_tr_s,y_te_s = train_test_split(x_tr, y_tr, test_size=0.20, random_state=0)
#print(x_trr[0])

In [0]:
#print(x_tr_ma.shape)

In [0]:
from sklearn import svm
model_pr = svm.SVC()
model_pr.fit(x_tr_s,y_tr_s)
model_pr.fit(x_tr_s,y_tr_s)
from sklearn import metrics
pr_te = model_pr.predict(x_te_s)


In [122]:
print("Accuraccy of model is: ",metrics.accuracy_score(y_te_s,pr_te))

Accuraccy of model is:  0.6973557692307693


In [16]:
from sklearn.ensemble import RandomForestClassifier
model_3=RandomForestClassifier()
model_3.fit(x_tr_s,y_tr_s)
pr_te_3 = model_3.predict(x_te_s)
print("Accuraccy of model is: ",metrics.accuracy_score(y_te_s,pr_te_3))


Accuraccy of model is:  0.7


In [118]:
from sklearn import linear_model
model_4=linear_model.LogisticRegression()
model_4.fit(x_tr_s,y_tr_s)
pr_te_4 = model_4.predict(x_te_s)
print("Accuraccy of model is: ",metrics.accuracy_score(y_te_s,pr_te_4))



Accuraccy of model is:  0.7125


In [119]:
from sklearn import tree
model_5=tree.DecisionTreeClassifier()
model_5.fit(x_tr_s,y_tr_s)
pr_te_5 = model_5.predict(x_te_s)
print("Accuraccy of model is: ",metrics.accuracy_score(y_te_s,pr_te_5))



Accuraccy of model is:  0.6305288461538462


In [121]:
from gensim.models import doc2vec
from collections import namedtuple

docs = []
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
ma=-1
for i, lis in enumerate(n_txt_tkn):
    words = lis
    ma=max(ma,i)
    tags = [i]
    docs.append(analyzedDocument(words, tags))

model_d2v = doc2vec.Doc2Vec(docs, size=300, window=8, min_count=1)



/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [0]:
x_d=[]
for i in range(len(n_txt)):
  x_d.append(model_d2v[i])
x_d = np.array(x_d)


In [0]:
x_tr_d,x_te_d,y_tr_d,y_te_d=train_test_split(x_d, y_tr, test_size=0.20, random_state=0)

#Normalising training data
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
x_tr_d = min_max_scaler.fit_transform(x_tr_d)

#Normalising Test data
x_te_d= min_max_scaler.transform(x_te_d)

In [125]:
model_pr_d = svm.SVC()
model_pr_d.fit(x_tr_d,y_tr_d)
from sklearn import metrics
pr_te_d = model_pr_d.predict(x_te_d)
print("Accuraccy of model is: ",metrics.accuracy_score(y_te_d,pr_te_d))

Accuraccy of model is:  0.9550480769230769


In [126]:
model_3_d=RandomForestClassifier()
model_3_d.fit(x_tr_d,y_tr_d)
pr_te_3_d = model_3_d.predict(x_te_d)
print("Accuraccy of model is: ",metrics.accuracy_score(y_te_d,pr_te_3_d))

Accuraccy of model is:  0.9497596153846154


In [127]:
model_4_d=linear_model.LogisticRegression()
model_4_d.fit(x_tr_d,y_tr_d)
pr_te_4_d = model_4_d.predict(x_te_d)
print("Accuraccy of model is: ",metrics.accuracy_score(y_te_s,pr_te_4_d))

Accuraccy of model is:  0.9661057692307692


In [128]:
from sklearn import tree
model_5_d=tree.DecisionTreeClassifier()
model_5_d.fit(x_tr_d,y_tr_d)
pr_te_5_d = model_5_d.predict(x_te_d)
print("Accuraccy of model is: ",metrics.accuracy_score(y_te_d,pr_te_5_d))



Accuraccy of model is:  0.9028846153846154


In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
image_ordering_dim="th"
model_cnn=Sequential()
model_cnn.add(Convolution2D(filters = 32, kernel_size = (3, 3), activation='relu',
                 input_shape = (100,50,1)))
model_cnn.add(MaxPooling2D(pool_size=(2,1)))
model_cnn.add(Convolution2D(filters = 32, kernel_size = (3, 3), activation='relu',
                 input_shape = (100,50,1)))
model_cnn.add(Convolution2D(filters = 32, kernel_size = (3, 3), activation='relu',
                 input_shape = (100,50,1)))
model_cnn.add(Convolution2D(filters = 32, kernel_size = (3, 3), activation='relu',
                 input_shape = (100,50,1)))
model_cnn.add(Dense(100))
model_cnn.add(Convolution2D(filters = 32, kernel_size = (3, 3), activation='relu',
                 input_shape = (100,50,1)))
model_cnn.add(Dense(100))
model_cnn.add(Convolution2D(filters = 32, kernel_size = (3, 3), activation='relu',
                 input_shape = (100,50,1)))
model_cnn.add(Dense(100))
model_cnn.add(Convolution2D(filters = 32, kernel_size = (3, 3), activation='relu',
                 input_shape = (100,50,1)))

model_cnn.add(Convolution2D(filters = 32, kernel_size = (3, 3), activation='relu',
                 input_shape = (100,50,1)))

model_cnn.add(Convolution2D(filters = 32, kernel_size = (3, 3), activation='relu',
                 input_shape = (100,50,1)))

#model_cnn.add(MaxPooling2D(pool_size=(2,1)))
model_cnn.add(Dropout(0.25))
model_cnn.add(Flatten())
model_cnn.add(Dense(100))
model_cnn.add(Dropout(0.25))
model_cnn.add(Dense(10))

model_cnn.add(Dense(1))
model_cnn.add(Activation('softmax'))
model_cnn.compile(loss='binary_crossentropy', optimizer = 'adadelta', metrics=["accuracy"])

In [ ]:
model_cnn.summary()

In [ ]:
#x_trr_2=x_trr.reshape(-1,400,1,1)
#print(type(x_trr_2))
y_trr_2=np.array(y_trr)
model_cnn.fit(x_trr,y_trr_2,nb_epoch=1)

In [ ]:
x_te_2=x_te.reshape(-1,100,50,1)
y_te_2=np.array(y_te)
model_cnn.evaluate(x_te_2,y_te_2)

In [ ]:
x_tr_s=np.array(x_tr_s)
x_tr_ls=x_tr_s.reshape(-1,1,50)

from keras.layers import Dense, LSTM, SimpleRNN

model_ls = Sequential()
#model_ls.add(LSTM(1,input_shape=(1,50), kernel_initializer='lecun_uniform', activation='relu', bias_regularizer=None, activity_regularizer=None, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
#model_ls.add(Dense(50))
model_ls.add(LSTM(1, input_shape=(1,50), kernel_initializer='lecun_uniform', activation='relu', bias_regularizer=None, activity_regularizer=None, dropout=0.2, recurrent_dropout=0.2))#, return_sequences=True))
model_ls.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
print(model_ls.summary())



In [ ]:
model_ls.fit(x_tr_ls, y_trr_2, epochs=20, batch_size=100)